In [180]:
import re
import os
import numpy as np
import pandas as pd
from pprint import pprint
from operator import itemgetter

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
import nltk
stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

musings_path = 'sam_musings/'
musings = os.listdir(musings_path)

# Import Dataset
print('Loading musings...')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
musings = '\n'.join([open(musings_path + musing).read() for musing in musings])
musing_sentences = tokenizer.tokenize(musings)
df = pd.DataFrame([{'content': sentence} for sentence in musing_sentences])

# Convert to list
data = df.content.values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        # deacc=True removes punctuations

data_words = list(sent_to_words(data))

# Build the bigram and trigram models
print('Making bigrams...')
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

print('Making trigrams...')
#trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
print('Modding n-grams...')
bigram_mod = gensim.models.phrases.Phraser(bigram)
#trigram_mod = gensim.models.phrases.Phraser(trigram)

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        try:
            doc = nlp(" ".join(sent)) 
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        except TypeError:
            if len(sent) == 0:
                print 'Null bigram found'
            else:
                print 'something is wrong with bigram: {}'.format(sent)
    return texts_out

# Remove Stop Words
print('Destroying stop words...')
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
print('Forming bigrams...')
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
print('Building spacy NLP parser...')
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
print('Lemmatizing bigrams...')
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
print('Building dictionary...')
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
print('Making corpus...')
corpus = [id2word.doc2bow(text) for text in texts]

topic_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100, 200]
lda_models = [] * len(topic_sizes)
coherence_model_ldas = [] * len(topic_sizes)
coherence_lda = np.zeros(len(topic_sizes))

# Build LDA model
print('Generating LDA Model...')
#for index, topics in enumerate(topic_sizes):
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                   id2word=id2word,
                                                   num_topics=50,
                                                   random_state=100,
                                                   update_every=1,
                                                   chunksize=100,
                                                   passes=10,
                                                   alpha='auto',
                                                   per_word_topics=True)
    
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Generate mallet topic
mallet_path = '../mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)


def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values


limit=150; start=50; step=10;

# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=start, limit=limit, step=step)

# Show graph
x = range(start, limit, step)

# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

# Select the model and print the topics
optimal_model = model_list[np.argmax(coherence_values)]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic


Loading musings...
Making bigrams...
Making trigrams...
Modding n-grams...
Destroying stop words...
Forming bigrams...
Building spacy NLP parser...
Lemmatizing bigrams...
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
N

Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigram found
Null bigra

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,3,0.0931,"issue, address, raise, discuss, accessibility,...","On Tuesday at lunch, I'm part of a small group..."
1,1,44,0.0247,"people, feel, remind, treat, refer, differentl...",I think this is one of those times that I need...
2,2,45,0.0266,"watch, video, drive, love, trouble, precisely,...",I hope I don't get myself in too much trouble.
3,3,3,0.0256,"issue, address, raise, discuss, accessibility,...",Shared governance is a core aspect of higher e...
4,4,30,0.0360,"faculty, member, staff, administrator, tenure,...",Its basic premise should be simple: The facult...
5,5,127,0.0393,"make, difference, decision, choice, happy, hug...","That is, they make decisions collaboratively w..."
6,6,125,0.0719,"faculty, committee, handbook, salary, represen...","At times, collaboration may take the form on c..."
7,7,125,0.0434,"faculty, committee, handbook, salary, represen...",Shared governance usually requires some form o...
8,8,125,0.0671,"faculty, committee, handbook, salary, represen...","That is, because it is complicated to delibera..."
9,9,121,0.0379,"student, group, current, parent, prospective, ...",Those representative groups then have a respon...


In [181]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0,0.1119,"student, academic, support, advisor, advise, a...",I really appreciate our alumni efforts.
1,1,0.1941,"pay, college, cost, close, full, worth, admiss...","If a file has read permission, you can look at..."
2,2,0.1648,"paper, free, conference, author, presentation,...",No.
3,3,0.0931,"issue, address, raise, discuss, accessibility,...","On Tuesday at lunch, I'm part of a small group..."
4,4,0.0931,"time, spend, log, effort, weekend, consume, sp...",[4] We have been approved to hire a fifth tenu...


In [69]:
sent_topics_sorteddf_mallet['Text'][1]

'CSC 321 provides an overview of practices, principles, and tools.'

In [160]:
def getTopicForQuery(question, model=lda_model):
    temp = question.lower()

    words = re.findall(r'\w+', temp, flags = re.UNICODE | re.LOCALE)
    
    important_words = [word for word in simple_preprocess(str(words)) if word not in stop_words]
    
    dictionary = corpora.Dictionary.load('sam_dict.dict')

    ques_vec = dictionary.doc2bow(important_words)

    topic_vec = model[[ques_vec]]

    likely_topic = max(topic_vec[0], key=itemgetter(1))[0]
    
    return likely_topic

In [190]:
query_dominant_topic = getTopicForQuery("Why don't you like potatoes?", model=optimal_model)
related_sentences = df_dominant_topic[df_dominant_topic['Dominant_Topic'] == query_dominant_topic].reset_index(drop=True)
related_sentences.loc[random.randint(0,len(related_sentences))]['Text']

'I think everyone should try CS.'

In [186]:
id2word.save('sam_dict.dict')

In [97]:
lda_model.get_topics().shape

(30, 1324)

In [55]:
df.loc[4].content

'On Friday, I was reading the Scarlet and Black, Grinnell\'s weekly newspaper [1].  In it was a short interview with the Overcoats [2], a band who are playing Grinnell on November 16.  The first question is "What kind of music did you both grow up listening to?"  Hana Elion\'s response starts "I liked the classics like [...]". Now, that\'s a phrase that many pop and rock musicians say.  In the 60\'s, the classics typically referred to artists like Elvis, Howlin\' Wolf, Hank Williams, Junior Parker and such.  By the 70\'s, the classics were artists like The Beatles, The Stones, The Who, Motown, Otis, James Brown, Dylan, Aretha and perhaps even the Velvets.  Moving forward a decade or two, one would probably add artists like Springsteen, Fleetwood Mac, Prince, and Michael Jackson.  I\'m not sure that anyone ever referred to the late 1970\'s punk and new wave acts as "classics".  However, by this time, it seems that Patti Smith, the Ramones, Hsker D, the Jam, the Beat, and other such acts

In [63]:
[[score for topic_id, score in topic] for topic in [doc for doc in lda_corpus]]

                                               content
0    On Tuesday at lunch, I'm part of a small group...
1    I think this is one of those times that I need...
2       I hope I don't get myself in too much trouble.
3    Shared governance is a core aspect of higher e...
4    Its basic premise should be simple: The facult...
5    That is, they make decisions collaboratively w...
6    At times, collaboration may take the form on c...
7    Shared governance usually requires some form o...
8    That is, because it is complicated to delibera...
9    Those representative groups then have a respon...
10    Successful shared governance is open governance.
11   Representative groups cannot communicate the w...
12   Whenever possible (and it's not always possibl...
13   Admittedly, shared governance includes some ac...
14   Ideally, in both case that "ownership" is stil...
15   I am concerned about the current status of sha...
16                                                Why?
17   First

In [145]:
lda_model.print_topic(29)

u'0.166*"good" + 0.126*"think" + 0.070*"well" + 0.065*"note" + 0.053*"help" + 0.039*"problem" + 0.020*"session" + 0.020*"postscript" + 0.020*"family" + 0.020*"fortune"'

In [166]:
from operator import itemgetter
import random

In [157]:
df_dominant_topic

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,23,0.0940,"make, shared_governance, form, discuss, chair,...","On Tuesday at lunch, I'm part of a small group..."
1,1,4,0.0533,"question, tutorial, recent, excellent, focus, ...",I think this is one of those times that I need...
2,2,13,0.0424,"goa, department, aspect, sit, talk, wait, rebe...",I hope I don't get myself in too much trouble.
3,3,23,0.0421,"make, shared_governance, form, discuss, chair,...",Shared governance is a core aspect of higher e...
4,4,15,0.0497,"change, policy, isc, hear, administration, sig...",Its basic premise should be simple: The facult...
5,5,30,0.0421,"good, campus, problem, teaching, colleague, ba...","That is, they make decisions collaboratively w..."
6,6,9,0.0562,"faculty, topic, small, stuff, interesting, str...","At times, collaboration may take the form on c..."
7,7,23,0.0764,"make, shared_governance, form, discuss, chair,...",Shared governance usually requires some form o...
8,8,10,0.0857,"group, people, discipline, grinnell, research,...","That is, because it is complicated to delibera..."
9,9,10,0.0553,"group, people, discipline, grinnell, research,...",Those representative groups then have a respon...


In [191]:
optimal_model.save('sam_lda_model.mm')